In [ ]:
# This is literally just a dataset from Kaggle. Will be used as main data.
import kagglehub
path = kagglehub.dataset_download("dster/rocket-league-rlcs-timeseries")
print(path)

100%|██████████| 14.1G/14.1G [08:57<00:00, 28.2MB/s]  

Extracting files...


/home/itagizade/.cache/kagglehub/datasets/dster/rocket-league-rlcs-timeseries/versions/4


In [ ]:
import shutil
from pathlib import Path

source = path+'rocket-league-rlcs-season-2025-replays/'
dest = Path().absolute()

shutil.move(source+'frames.parquet', dest)
shutil.move(source+'games.parquet', dest)

NameError: name 'path' is not defined

In [ ]:
# --- Enrichment Dataset collection ---
REGIONS = {
    "Europe": "https://liquipedia.net/rocketleague/Rocket_League_Championship_Series/2025/Rankings/Europe",
    "North_America": "https://liquipedia.net/rocketleague/Rocket_League_Championship_Series/2025/Rankings/North_America",
    "South_America": "https://liquipedia.net/rocketleague/Rocket_League_Championship_Series/2025/Rankings/South_America",
    "Oceania": "https://liquipedia.net/rocketleague/Rocket_League_Championship_Series/2025/Rankings/Oceania",
    "MENA": "https://liquipedia.net/rocketleague/Rocket_League_Championship_Series/2025/Rankings/Middle_East_and_North_Africa",
    "APAC": "https://liquipedia.net/rocketleague/Rocket_League_Championship_Series/2025/Rankings/Asia-Pacific",
    "SSA": "https://liquipedia.net/rocketleague/Rocket_League_Championship_Series/2025/Rankings/Sub-Saharan_Africa"
}

# Identity header to avoid 403 Forbidden
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0",
    "Accept-Language": "en-US,en;q=0.9"
}

all_data = []

print("--- Starting Global Ranking Collection (BeautifulSoup) ---")

for region_name, url in REGIONS.items():
    print(f"Fetching {region_name}...", end=" ", flush=True)
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed (Status: {response.status_code})")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 1. Find rows that belong to the tabbed content
        # These rows have the attribute 'data-toggle-area-content'
        rows = soup.find_all('tr', attrs={'data-toggle-area-content': True})
        
        if not rows:
            print("Warning: No ranking rows found (Layout might have changed).")
            continue

        # 2. Extract & De-duplicate
        # We use a dictionary to store {TeamName: MaxPoints}
        # This handles the issue where teams appear multiple times in history tabs
        region_teams = {}
        
        for row in rows:
            cols = row.find_all('td')
            
            # Ensure row has enough columns (Rank, Trend, Team, Points, ...)
            if len(cols) < 4: 
                continue 
            
            # --- Extract Team Name (Index 2) ---
            # Try to get text from the specific team template class first
            team_cell = cols[2]
            team_node = team_cell.find(class_='team-template-text')
            if team_node:
                team_name = team_node.get_text(strip=True)
            else:
                team_name = team_cell.get_text(strip=True)
            
            # --- Extract Points (Index 3) ---
            points_cell = cols[3]
            points_text = points_cell.get_text(strip=True)
            
            # Clean numeric data
            points = 0
            if points_text and points_text != "-":
                try:
                    # Remove non-digits (like footnote markers)
                    clean_digits = ''.join(filter(str.isdigit, points_text))
                    points = int(clean_digits) if clean_digits else 0
                except ValueError:
                    points = 0
            
            # Logic: Keep the highest point total seen for this team
            if team_name:
                if team_name not in region_teams:
                    region_teams[team_name] = points
                else:
                    if points > region_teams[team_name]:
                        region_teams[team_name] = points

        # 3. Add to master list
        for team, pts in region_teams.items():
            all_data.append({
                'Team': team,
                'Points': pts,
                'Region': region_name
            })
            
        print(f"Success! Parsed {len(region_teams)} unique teams.")

    except Exception as e:
        print(f"Error: {e}")
    
    # Be polite to the server
    time.sleep(2)

# --- SAVE ---
if all_data:
    df = pd.DataFrame(all_data)
    
    # Optional: Filter out teams with 0 points to keep dataset clean
    df = df[df['Points'] > 0]
    
    # Sort for better readability
    df = df.sort_values(by=['Region', 'Points'], ascending=[True, False])
    
    filename = "rl_rankings.csv"
    df.to_csv(filename, index=False)
    
    print(f"\nCompleted! Saved {len(df)} ranked teams to '{filename}'.")
    print(df.head())
    print("\nSample per region:")
    print(df.groupby('Region').size())
else:
    print("\nNo data collected.")

--- Starting Global Ranking Collection (BeautifulSoup) ---
Fetching Europe... Success! Parsed 28 unique teams.
Fetching North_America... Success! Parsed 30 unique teams.
Fetching South_America... Success! Parsed 27 unique teams.
Fetching Oceania... Success! Parsed 43 unique teams.
Fetching MENA... Success! Parsed 43 unique teams.
Fetching APAC... Success! Parsed 36 unique teams.
Fetching SSA... Success! Parsed 37 unique teams.

Completed! Saved 240 ranked teams to 'rl_rankings.csv'.
                 Team  Points Region
171        Virtus.pro     109   APAC
172      Movistar KOI      66   APAC
173      Dreamstation      43   APAC
177  KINOTROPE gaming      37   APAC
174         God Speed      27   APAC

Sample per region:
Region
APAC             36
Europe           28
MENA             42
North_America    30
Oceania          40
SSA              37
South_America    27
dtype: int64
